In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


# Initialize the WebDriver
driver = webdriver.Edge()
driver.get('https://printyourcopy.com/free-payslip-generator')
time.sleep(1)

# Function to clear input fields for the next iteration
def clear_form_fields():
    """
    Clears all input fields on the form for earnings and deductions, 
    as well as the employee details section.
    """
    # Clear employee details section
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.CONTROL + "a")  # Select all text
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.DELETE)        # Clear it
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(Keys.TAB)  
    
    # Clear earnings field
    for i in range(6):
        time.sleep(0.2)
        xpath = f'//*[@id="earnings.{i}.amount"]'
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)  

    # Clear deductions fields
    for i in range(4):
        time.sleep(0.2)
        xpath = f'//*[@id="deductions.{i}.amount"]'
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)  

# Function to preprocess data from the Excel sheet
def preprocess_employee_data(data):
    """
    Reads and preprocesses employee data from an Excel file.

    :param file_path: Path to the Excel file
    :return: A pandas DataFrame with the processed data
    """
    df = pd.read_excel(data)
    df['Date Of Joining'] = pd.to_datetime(df['Date Of Joining']).dt.strftime('%Y-%m-%d')
    columns_to_convert = ['Gross Salary', 'Bonus / Commission', 'Increment', 
                        'Reimbursment Amount', 'Compensation', 'Adjustments', 
                        'Absents Deduction', 'Lates Deduction']
    for col in columns_to_convert:
        df[col] = df[col].astype(int)

    return df

# Load and preprocess employee data
df = preprocess_employee_data("Demo.xlsx")

# Function to populate static details on the form
def populate_static_details():
    """
    Populates static details such as employer details, currency, and adds additional earning/deduction fields.
    """
    # Upload logo
    logo = os.path.abspath("Logo.jpg")
    driver.find_element(By.XPATH, '//*[@id="logo"]').send_keys(logo)

    # Input employer details
    employer_details = f"""
    Robust Support & Solutions
    Office No.404A, Fortune Tower
    PECHS Block 6, Karachi, Pakistan
    Phone & WhatsApp: 0311-3859635
    """
    driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea').send_keys(employer_details)
    
    # Clear and populate other static fields
    static_fields_xpaths  = [
        '//*[@id="serial"]', '//*[@id="employeeSign"]', 
        '//*[@id="employerSign"]', '//*[@id="currency"]'
    ]
    for xpath in static_fields_xpaths :
        driver.find_element(By.XPATH, xpath).send_keys(Keys.CONTROL + "a")  # Select all text
        driver.find_element(By.XPATH, xpath).send_keys(Keys.DELETE)        # Clear it
        if xpath == '//*[@id="currency"]':
            driver.find_element(By.XPATH, xpath).send_keys("PKR")
        driver.find_element(By.XPATH, xpath).send_keys(Keys.TAB)   
        time.sleep(0.2)
    
    # Add extra fields for earnings and deductions
    n = 4
    m = 11
    for i in range(4):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button').click()
        n+=1
    for i in range(3):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{m}]/td/button').click()
        m+=1

# Populate static details
populate_static_details()
time.sleep(0.5)

# Function to update field descriptions
def update_field_descriptions(driver, field_xpath, description, clear_existing=False):
    """
    Updates the description fields for earnings or deductions.

    :param fields: List of tuples containing field IDs, descriptions, and a clear flag
    """
    element = driver.find_element(By.XPATH, f'//*[@id="{field_xpath}"]')
    if clear_existing:
        element.send_keys(Keys.CONTROL + "a")  # Select all text
        element.send_keys(Keys.DELETE)        # Clear it
    element.send_keys(description)
    element.send_keys(Keys.TAB)
    time.sleep(0.2)  # Sleep to allow UI updates

# Update descriptions for earnings and deductions
earnings_descriptions = [
    ("earnings.0.description", "Gross Salary", True),  # Requires clearing
    ("earnings.1.description", "Bonus / Commission", True),  # Requires clearing
    ("earnings.2.description", "Reimbursement", False),
    ("earnings.3.description", "Increment", False),
    ("earnings.4.description", "Compensation", False),
    ("earnings.5.description", "Adjustments", False)
]
deductions_descriptions = [
    ("deductions.0.description", "Absents", True),  # Requires clearing
    ("deductions.1.description", "Lates", False),
    ("deductions.2.description", "Payroll Tax", False),
    ("deductions.3.description", "Others", False)
]

# Update earnings descriptions
for field_xpath, description, clear in earnings_descriptions:
    update_field_descriptions(driver, field_xpath, description, clear_existing=clear)

# Update deductions descriptions
for field_xpath, description, clear in deductions_descriptions:
    update_field_descriptions(driver, field_xpath, description, clear_existing=clear)

# Function to input employee-specific details and generate payslips
def generate_payslips():
    """
    Iterates over the DataFrame to input employee details, generate payslips,
    and save the files with appropriate names.
    """
    for i in range(df.shape[0]):
        # Populate employee details
        employee_details= f"""{df.loc[i, 'Full Name (As Per CNIC)']}
        {df.loc[i, 'CNIC Number']}
        {df.loc[i, 'Designation']}
        DOJ: {df.loc[i, 'Date Of Joining']}
        """
        driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea').send_keys(employee_details)

        driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(str(df.loc[i, 'Gross Salary']))
        driver.find_element(By.XPATH, '//*[@id="earnings.0.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(str(df.loc[i, 'Bonus / Commission']))
        driver.find_element(By.XPATH, '//*[@id="earnings.1.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(str(df.loc[i, 'Reimbursment Amount']))
        driver.find_element(By.XPATH, '//*[@id="earnings.2.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(str(df.loc[i, 'Increment']))
        driver.find_element(By.XPATH, '//*[@id="earnings.3.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(str(df.loc[i, 'Compensation']))
        driver.find_element(By.XPATH, '//*[@id="earnings.4.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(str(df.loc[i, 'Adjustments']))
        driver.find_element(By.XPATH, '//*[@id="earnings.5.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(str(df.loc[i, 'Absents Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.0.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(str(df.loc[i, 'Lates Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.1.amount"]').send_keys(Keys.TAB)  
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(str(df.loc[i, 'Payroll Tax Deduction']))
        driver.find_element(By.XPATH, '//*[@id="deductions.2.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(str(df.loc[i, 'Other Deductions']))
        driver.find_element(By.XPATH, '//*[@id="deductions.3.amount"]').send_keys(Keys.TAB) 
        time.sleep(0.3)

        # Check if the file exists and remove it if it does
        original_file = r"C:\Users\Desktop\Downloads\Payslip.pdf"
        if os.path.exists(original_file):
            os.remove(original_file)
            print(f"Removed previous Payslip file!")

        # Generate and rename the payslip
        driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[2]/button[2]').click()
        time.sleep(2)
        new_file = fr'Payslips\{df.loc[i, "Full Name (As Per CNIC)"]} - {df.loc[i, "Month"]}.pdf'
        os.rename(original_file, new_file)
        print(f"File Directory change and Renamed: {original_file} -> {new_file}")
        
        # Clear form for the next employee
        clear_form_fields()
        time.sleep(0.3)

# Generate payslips for all employees
generate_payslips()

File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Haris Dilshad - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Fahad Bin Zahid - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Ibrahim Najmuddin - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Zuhaib Ghori - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Sabih Uddin Farooqi - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Muhammad Waqas Farooqi - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Rhonda Sacrafamilia - OCTOBER' 2024.pdf
File Directory change and Renamed: C:\Users\Desktop\Downloads\Payslip.pdf -> Payslips\Amir Bin Nasim - OCTOBER' 2024

**^ Taking Approx 15 mins to process 73 payslips**

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# Initialize WebDriver
driver = webdriver.Edge()
driver.get('https://printyourcopy.com/free-payslip-generator')
wait = WebDriverWait(driver, 10)

# Data Preprocessing Function
def data_preprocessing(file_path):
    df = pd.read_excel(file_path)
    df['Date Of Joining'] = pd.to_datetime(df['Date Of Joining']).dt.strftime('%Y-%m-%d')
    columns_to_convert = ['Gross Salary', 'Bonus / Commission', 'Increment',
                          'Reimbursment Amount', 'Compensation', 'Adjustments',
                          'Absents Deduction', 'Lates Deduction']
    df[columns_to_convert] = df[columns_to_convert].astype(int)
    return df

df = data_preprocessing("Demo.xlsx")

# Function to clear values from input fields
def clear_fields(xpaths):
    for xpath in xpaths:
        field = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
        field.send_keys(Keys.CONTROL + "a", Keys.DELETE)

def clear_values():
    # Clear employee details and earnings/deductions
    employee_details_xpath = '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea'
    clear_fields([employee_details_xpath] +
                 [f'//*[@id="earnings.{i}.amount"]' for i in range(6)] +
                 [f'//*[@id="deductions.{i}.amount"]' for i in range(4)])

# Static Setup Function
def setup_static_details():
    # Upload Logo
    logo_path = os.path.abspath("Logo.jpg")
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="logo"]'))).send_keys(logo_path)
    
    # Fill Employer Details
    employer_details = """Robust Support & Solutions
Office No.404A, Fortune Tower
PECHS Block 6, Karachi, Pakistan
Phone & WhatsApp: 0311-3859635"""
    wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[1]/textarea'))).send_keys(employer_details)
    
    # Clear and set serial, signatures, and currency fields
    fields_to_clear = ['//*[@id="serial"]', '//*[@id="employeeSign"]', '//*[@id="employerSign"]', '//*[@id="currency"]']
    clear_fields(fields_to_clear)
    driver.find_element(By.XPATH, '//*[@id="currency"]').send_keys("PKR", Keys.TAB)

    # Add extra earning and deduction fields
    n = 4
    m = 11
    for i in range(4):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{n}]/td/button').click()
        n+=1
    for i in range(3):
        time.sleep(0.5)
        driver.find_element(By.XPATH, f'/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[2]/table/tbody/tr[{m}]/td/button').click()
        m+=1

setup_static_details()

# Function to update descriptions
def update_field_descriptionss(descriptions, category):
    for field_id, text, clear in descriptions:
        element = wait.until(EC.presence_of_element_located((By.XPATH, f'//*[@id="{field_id}"]')))
        if clear:
            element.send_keys(Keys.CONTROL + "a", Keys.DELETE)
        element.send_keys(text, Keys.TAB)

# Update earning and deduction descriptions
earnings_descriptions = [
    ("earnings.0.description", "Gross Salary", True),
    ("earnings.1.description", "Bonus / Commission", True),
    ("earnings.2.description", "Reimbursement", False),
    ("earnings.3.description", "Increment", False),
    ("earnings.4.description", "Compensation", False),
    ("earnings.5.description", "Adjustments", False)
]
deductions_descriptions = [
    ("deductions.0.description", "Absents", True),
    ("deductions.1.description", "Lates", False),
    ("deductions.2.description", "Payroll Tax", False),
    ("deductions.3.description", "Others", False)
]

update_field_descriptionss(earnings_descriptions, "earnings")
update_field_descriptionss(deductions_descriptions, "deductions")

# Main Function to Input Values and Generate Payslips
def process_payslips():
    download_dir = r"C:\Users\Desktop\Downloads"
    output_dir = "Payslips"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in df.iterrows():
        # Fill employee details
        employee_details = f"""{row['Full Name (As Per CNIC)']}
{row['CNIC Number']}
{row['Designation']}
DOJ: {row['Date Of Joining']}"""
        wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[1]/div[1]/div[4]/div[2]/textarea'))).send_keys(employee_details)

        # Fill earnings and deductions
        fields = {
            'earnings': ['Gross Salary', 'Bonus / Commission', 'Reimbursment Amount', 'Increment', 'Compensation', 'Adjustments'],
            'deductions': ['Absents Deduction', 'Lates Deduction', 'Payroll Tax Deduction', 'Other Deductions']
        }
        for i, col in enumerate(fields['earnings']):
            driver.find_element(By.XPATH, f'//*[@id="earnings.{i}.amount"]').send_keys(str(row[col]), Keys.TAB)
        for i, col in enumerate(fields['deductions']):
            driver.find_element(By.XPATH, f'//*[@id="deductions.{i}.amount"]').send_keys(str(row[col]), Keys.TAB)

        # Download Payslip
        original_file = os.path.join(download_dir, "Payslip.pdf")
        if os.path.exists(original_file):
            os.remove(original_file)
        driver.find_element(By.XPATH, '/html/body/div[1]/main/div[2]/div/section[1]/div/div[2]/button[2]').click()
        time.sleep(2)

        # Rename and move the file
        new_file = os.path.join(output_dir, f"{row['Full Name (As Per CNIC)']} - {row['Month']}.pdf")
        if os.path.exists(original_file):
            os.rename(original_file, new_file)
            print(f"Saved Payslip: {new_file}")
        
        clear_values()

process_payslips()


**^ Taking Approx 8-9 mins to process all 73 payslips**

**Key Changes:**
 - Dynamic Waiting: Used WebDriverWait instead of time.sleep for stability.
 - Reusable Logic: Refactored repetitive tasks like clearing fields, filling descriptions, and processing data into functions.
 - Optimized function: Refactored the main generate_payslips function to improve efficiency by minimizing redundant code, dynamically handling   data inputs, and leveraging optimized logic for better performance.

This optimization ensures better maintainability and robustness in handling different scenarios.